In [1]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
# Load English tokenizer, tagger,  
# parser, NER and word vectors 
# nlp = spacy.load("en_core_web_sm") 

In [3]:
example = 'Democratic Sen. Mazie Hirono blasted “the men in this country,” blaming them for sexual assault and urged them to “just shut up and step up” amid allegations of misconduct against President Trump’s Supreme Court nominee Judge Brett Kavanaugh.\n\nAt a press conference Tuesday evening, the Hawaii Democrat made the sweeping generalization against “the men” in the U.S. for “perpetuating” sexual assault.\n\n“I expect the men in this country and the men on this committee, because we all signed onto this letter, to demand an FBI investigation,” Hirono said, referring to a request that the FBI investigate the claims made by Kavanaugh’s accuser, Christine Blasey Ford.\n\nTRUMP SAYS \'WE\'LL HAVE TO MAKE A DECISION\' ON KAVANAUGH\'S FUTURE IF ACCUSER GIVES \'CREDIBLE\' TESTIMONY\n\n“But really, guess who is perpetuating all of these kinds of actions? It’s the men in this country. And I just want to say to the men in this country: just shut up and step up, do the right thing for a change,” Hirono said.\n\n“Not only do women, like Dr. Ford, who bravely comes forward, need to be heard, but they need to be believed,” Hirono added. “We cannot continue the victimization and the smearing of someone like Dr. Ford.”\n\nFord’s identity was revealed in a Washington Post article published over the weekend, following an anonymous letter obtained by Sen. Dianne Feinstein, D-Calif., alleging sexual assault. The California-based professor said that Kavanaugh, 36 years ago while in high school, pinned her down, tried to remove her bathing suit and put his hand over her mouth when she attempted to scream. Ford alleged that she was in the room with Kavanaugh and another man, who has been identified as Mark Judge.\n\nBoth Kavanaugh and Judge denied the allegations.\n\n“This is a completely false allegation. I have never done anything like what the accuser describes—to her or to anyone,” Kavanaugh said in a statement Monday. “Because this never happened, I had no idea who was making the accusation until she identified herself [Sunday.]”\n\nELIZABETH WARREN SLAMMED OVER EDITING OF KAVANAUGH VIDEO\n\nThe Senate Judiciary Committee invited both Ford and Kavanaugh to testify in either a public or private setting over the allegations.\n\nKavanaugh said he was “willing” to speak to the committee “in any way” deemed appropriate to “refute this false allegation” and to defend his “integrity.”\n\nFord’s attorney Monday suggested she would be willing to testify before the committee under oath, but has yet to officially accept the invitation to appear before the panel.\n\nFord has said she doesn’t want to appear until after an FBI investigation takes place.\n\nA highly-placed law enforcement source told Fox News on Wednesday that there will not be an FBI investigation into the allegations leveled at Kavanaugh.\n\nThe source told Fox News there cannot be a criminal investigation, because there is no crime alleged where the statute of limitations hasn\'t run. The source also said there are no allegations of a federal crime.\n\nKAVANAUGH ACCUSER CHRISTINE BLASEY FORD DEMANDS \'FULL INVESTIGATION\' BY FBI BEFORE TESTIFYING, IN LETTER FROM HER LAWYERS\n\nThe FBI only handles background check investigations, the source said, and should they find information, they would pass it along.\n\n"This is a political issue, not a law enforcement one," the source told Fox News.\n\nThe committee has already delayed Kavanaugh\'s confirmation vote, which was originally scheduled for Thursday. A hearing is now scheduled for Monday.\n\nSenate Judiciary Committee Chairman Chuck Grassley, R-Iowa, said that the hearing to examine the allegations set for Monday could be canceled if Ford does not accept the committee\'s invitation. Grassley said that his office has reached out several times to Ford and her attorneys to discuss her allegations, but has not received a response.\n\nFox News\' John Roberts contributed to this report.'

In [23]:


def collect_sources(article):
    doc = nlp(article)
    sentences = []
    sources = []
    sent = []
    for token in doc: 
        word = str(token)
        entity = str(token.ent_type_)
        pos = str(token.pos_)
        pair = [word, entity, pos]
        sent.append(pair)
        # construct sentences
        punc = ['.', '?', '!']
        if word in punc:
            sentences.append(sent)
            sent = []
    # loop through sentences constructing sources
    collection = []
    for s in sentences:
        co = []
        sources = []
        quote_bool = 0
        for idx, val in enumerate(s):
            indicators = ['said', 'added', 'told', 'made', 'blasted', 'suggested', 
                         'claimed', 'called', 'conceded', 'argued']
            if val[0] in indicators:
                co.append(val)
                for j in s[max(idx-6, 0):idx]:
                    if j[1] in {'NORP', 'ORG', 'PERSON'}:
                        co.append(j)
                    if j[2] in {'NOUN'}:
                        co.append(j)
            quotes = ['“' , '”' , '"']
            if val[0] in quotes:
                quote_bool = 1
        if quote_bool:
            co.append(['""', 'quotes', ''])
        if len(co)>=2:
            collection.append(co)     
    return collection

In [24]:
z = collect_sources(example)

In [25]:
len(z)

17

In [26]:
z

[[['blasted', '', 'VERB'],
  ['Democratic', 'NORP', 'ADJ'],
  ['Mazie', 'PERSON', 'PROPN'],
  ['Hirono', 'PERSON', 'PROPN'],
  ['""', 'quotes', '']],
 [['made', '', 'VERB'],
  ['evening', 'TIME', 'NOUN'],
  ['Democrat', 'NORP', 'PROPN'],
  ['""', 'quotes', '']],
 [['said', '', 'VERB'],
  ['FBI', 'ORG', 'PROPN'],
  ['investigation', '', 'NOUN'],
  ['Hirono', 'ORG', 'PROPN'],
  ['made', '', 'VERB'],
  ['FBI', 'ORG', 'PROPN'],
  ['claims', '', 'NOUN'],
  ['""', 'quotes', '']],
 [['said', '', 'VERB'],
  ['change', '', 'NOUN'],
  ['Hirono', 'ORG', 'PROPN'],
  ['""', 'quotes', '']],
 [['added', '', 'VERB'], ['Hirono', 'ORG', 'PROPN'], ['""', 'quotes', '']],
 [['said', '', 'VERB'], ['professor', '', 'NOUN']],
 [['said', '', 'VERB'],
  ['anyone', '', 'NOUN'],
  ['Kavanaugh', 'PERSON', 'PROPN'],
  ['""', 'quotes', '']],
 [['said', '', 'VERB'],
  ['Kavanaugh', 'PERSON', 'PROPN'],
  ['""', 'quotes', '']],
 [['suggested', '', 'VERB'],
  ['Ford', 'ORG', 'PROPN'],
  ['attorney', '', 'NOUN'],
  ['""'

In [27]:
ex2 = 'Senators nervously awaited the arrival of a new FBI report on sexual allegations that could make or break Brett Kavanaugh’s tottering Supreme Court nomination Wednesday as aggressive protesters and an unusually strong security response added to a feeling of high anxiety inside the U.S. Capitol.\n\nAs lawmakers anticipated the report, expected as early as Wednesday evening, three moderate GOP senators who could decide the conservative jurist’s fate rebuked President Donald Trump for mocking one accuser, Christine Blasey Ford, by mimicking her responses to questions at last week’s dramatic Senate Judiciary Committee hearing.\n\nTheir reactions left Republicans concerned that Trump had complicated their effort to cement Kavanaugh’s support in a chamber where the GOP holds a razor-thin 51-49 majority. Depending on when the FBI report arrived, Senate Majority Leader Mitch McConnell was expected to trigger a process that could lead to a crucial initial vote Friday and a climactic confirmation roll call over the weekend.\n\nInside the Capitol, mounting political strains over the approaching election-season showdown were mirrored by growing anxieties over senators’ security following frequent and at times aggressive demonstrations by anti-Kavanaugh protesters. Unusually large numbers of Capitol Hill Police officers restricted movements in corridors and formed wedges around senators walking through hallways. Some lawmakers also complained of being confronted outside their homes.\n\nOn the Senate floor, McConnell, R-Ky., claimed the protesters were “part of the organized effort” to derail Kavanaugh’s nomination and said, “There is no chance in the world that they’re going to scare us out of doing our duty.”\n\nSen. Susan Collins, R-Maine, told reporters that Trump’s Tuesday night lampooning of Ford at a Mississippi campaign rally was “just plain wrong.” Sen. Lisa Murkowski, R-Alaska, called it “wholly inappropriate and in my view unacceptable,” and Sen. Jeff Flake, R-Ariz., said on NBC’s “Today” show that the remarks were “kind of appalling.”\n\nThose GOP senators, along with Democrats Heidi Heitkamp of North Dakota and Joe Manchin of West Virginia, have yet to declare how they will vote on Kavanaugh. Other Republicans conceded that Trump’s insults could be damaging.\n\n“All of us need to keep in mind there’s a few people that are on the fence right now. And right now, that’s sort of where our focus needs to be,” said Sen. Bob Corker of Tennessee, who has traded barbs with Trump and will retire at year’s end.\n\nEven Trump ally Lindsey Graham, R-S.C., said at an event hosted by The Atlantic magazine: “I would tell him, knock it off. You’re not helping,”\n\nSenate Minority Leader Chuck Schumer, D-N.Y., said Trump’s insults of Ford marked a “new low.”\n\nThe California psychology professor has testified that a drunken Kavanaugh sexually abused her in a locked room at a high school party in the 1980s and has said she believed he was trying to rape her. Kavanaugh has denied her assertions and those of two other women, who have accused him of other instances of sexual misconduct in the 1980s.\n\nFord’s attorney complained Wednesday that the FBI has not contacted her for this week’s interviews. And Democrats argued that the investigation has been insufficient, lacking interviews with her, with Kavanaugh and others who Kavanaugh’s accusers have said could have knowledge about the alleged incidents.\n\nLawmakers said that once the FBI report arrived, senators and a small number of top aides would be allowed to read it in a secure room in the Capitol complex. Republicans have said they are working under an agreement governing background checks dating from the Obama administration, under which investigations are confidential and closely held.\n\nSens. Corker and Mike Rounds, R-S.D., said senators were expecting reports that FBI agents compile on their interviews with subjects, perhaps accompanied by a cover letter. But background checks do not traditionally contain investigators’ conclusions about who they believe is credible.\n\nWashington has been awaiting completion of the investigation since last week, when Flake, Collins and Murkowski pressured a reluctant Trump and GOP leaders to order the FBI to renew its background check of the 53-year-old Kavanaugh.\n\nThe FBI interviewed several people, including three who Ford has said attended a 1982 high school gathering in suburban Maryland where she says Kavanaugh’s attack occurred, plus another Kavanaugh friend. The agency has also spoken to a second woman, Deborah Ramirez, who has claimed Kavanaugh exposed himself to her at a Yale party when both were freshmen.\n\nWhile some senators from both parties have said they’d like at least a summary of the findings to be released, Senate procedures call for such checks to be kept confidential and it’s unclear what will be released, other than through leaks.\n\n“None of that stuff’s public,” Judiciary panel Chairman Chuck Grassley, R-Iowa, told reporters. “If you want people to be candid when they talk to the FBI, you ain’t going to make that public.”\n\nIn an interview, No. 2 Senate Democratic leader Richard Durbin of Illinois said McConnell was “hell bent” on confirming Kavanaugh this week.\n\nDemocrats also demanded that the FBI privately brief the Senate about the investigation before the chamber votes. McConnell rejected that request in a letter Wednesday to Schumer, saying Democrats would use it to delay Kavanaugh’s confirmation.\n\n___\n\nAssociated Press writers Eric Tucker, Michael Balsamo, Catherine Lucey, Zeke Miller, Padmananda Rama, Matthew Daly, Mary Clare Jalonick and Kevin Freking contributed from Washington. Will Weissert contributed from Austin, Texas.'

In [28]:
z1 = collect_sources(ex2)

In [29]:
len(z1)

19

In [30]:
z1

[[['added', '', 'VERB'], ['security', '', 'NOUN'], ['response', '', 'NOUN']],
 [['claimed', '', 'VERB'],
  ['said', '', 'VERB'],
  ['nomination', '', 'NOUN'],
  ['""', 'quotes', '']],
 [['told', '', 'VERB'],
  ['Collins', 'PERSON', 'PROPN'],
  ['R', 'ORG', 'PROPN'],
  ['-', 'ORG', 'PUNCT'],
  ['Maine', 'ORG', 'PROPN'],
  ['""', 'quotes', '']],
 [['called', '', 'VERB'],
  ['Murkowski', 'PERSON', 'PROPN'],
  ['""', 'quotes', '']],
 [['said', '', 'VERB'], ['""', 'quotes', '']],
 [['conceded', '', 'VERB'], ['Republicans', 'NORP', 'PROPN']],
 [['said', '', 'VERB'], ['focus', '', 'NOUN'], ['""', 'quotes', '']],
 [['said', '', 'VERB'], ['Graham', 'PERSON', 'PROPN'], ['""', 'quotes', '']],
 [['said', '', 'VERB'], ['Schumer', 'PERSON', 'PROPN'], ['""', 'quotes', '']],
 [['said', '', 'VERB'],
  ['party', '', 'NOUN'],
  ['1980s', 'DATE', 'NOUN'],
  ['""', 'quotes', '']],
 [['argued', '', 'VERB'],
  ['Democrats', 'NORP', 'PROPN'],
  ['said', '', 'VERB'],
  ['others', '', 'NOUN'],
  ['Kavanaugh', '

In [8]:
doc = nlp(example)
print([(X.text, X.label_) for X in doc.ents])
y = [(X.text, X.label_) for X in doc.ents]
hol = set()
for i in y:
    hol.add(i[1])

[('Democratic', 'NORP'), ('Mazie Hirono', 'PERSON'), ('Trump’s Supreme Court', 'ORG'), ('Brett Kavanaugh', 'PERSON'), ('Tuesday', 'DATE'), ('evening', 'TIME'), ('Hawaii', 'GPE'), ('Democrat', 'NORP'), ('U.S.', 'GPE'), ('FBI', 'ORG'), ('Hirono', 'ORG'), ('FBI', 'ORG'), ('Kavanaugh’s', 'ORG'), ('Christine Blasey Ford', 'PERSON'), ('KAVANAUGH', 'ORG'), ('Hirono', 'ORG'), ('Ford', 'PERSON'), ('Hirono', 'ORG'), ('Ford', 'PERSON'), ('Ford', 'ORG'), ('Washington Post', 'ORG'), ('the weekend', 'DATE'), ('Dianne Feinstein', 'PERSON'), ('D-Calif', 'NORP'), ('California', 'GPE'), ('Kavanaugh', 'ORG'), ('36 years ago', 'DATE'), ('Ford', 'ORG'), ('Kavanaugh', 'PERSON'), ('Mark Judge', 'PERSON'), ('Kavanaugh', 'PERSON'), ('Kavanaugh', 'PERSON'), ('Monday', 'DATE'), ('Sunday', 'DATE'), ('KAVANAUGH', 'ORG'), ('The Senate Judiciary Committee', 'ORG'), ('Ford', 'ORG'), ('Kavanaugh', 'ORG'), ('Kavanaugh', 'PERSON'), ('Ford', 'ORG'), ('Monday', 'DATE'), ('Ford', 'ORG'), ('FBI', 'ORG'), ('Fox News', 'ORG')

In [10]:
hol

{'DATE', 'GPE', 'NORP', 'ORG', 'PERSON', 'TIME'}

In [5]:
# Token and Tag 
for token in doc: 
    print(token, token.pos_, token.ent_type_) 
# token.tag_, token.dep_,
# You want list of Verb tokens 
#print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"]) 

Democratic ADJ NORP
Sen. PROPN 
Mazie PROPN PERSON
Hirono PROPN PERSON
blasted VERB 
“ PROPN 
the DET 
men NOUN 
in ADP 
this DET 
country NOUN 
, PUNCT 
” PUNCT 
blaming VERB 
them PRON 
for ADP 
sexual ADJ 
assault NOUN 
and CCONJ 
urged VERB 
them PRON 
to ADP 
“ PROPN 
just ADV 
shut VERB 
up PART 
and CCONJ 
step VERB 
up PART 
” PUNCT 
amid ADP 
allegations NOUN 
of ADP 
misconduct NOUN 
against ADP 
President PROPN 
Trump PROPN ORG
’s PROPN ORG
Supreme PROPN ORG
Court PROPN ORG
nominee NOUN 
Judge PROPN 
Brett PROPN PERSON
Kavanaugh PROPN PERSON
. PUNCT 


 SPACE 
At ADP 
a DET 
press NOUN 
conference NOUN 
Tuesday PROPN DATE
evening NOUN TIME
, PUNCT 
the DET 
Hawaii PROPN GPE
Democrat PROPN NORP
made VERB 
the DET 
sweeping ADJ 
generalization NOUN 
against ADP 
“ PUNCT 
the DET 
men NOUN 
” PUNCT 
in ADP 
the DET 
U.S. PROPN GPE
for ADP 
“ PROPN 
perpetuating VERB 
” PUNCT 
sexual ADJ 
assault NOUN 
. PUNCT 


 SPACE 
“ PROPN 
I PRON 
expect VERB 
the DET 
men NOUN 
in ADP 
t